In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import math
import cv2
%matplotlib inline

In [3]:
num_of_epochs = 3

In [4]:
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

if torch.cuda.is_available():
    device = torch.device('cuda')


False
True


## Read data

In [5]:
path_list = []
letter = []
dirName = './dataset_mnist'
for path, subdirs, files in os.walk(dirName):
    for name in files:
        letter.append(path[-3])
        if path[-3] == 'o':
            print(path)
        path_list.append(os.path.join(path, name))
        
        #dataset[os.path.join(path, name)] = path[-3]
        #print(os.path.join(path, name))
        #print(path[-3])
#print(dataset)
#print(path_list)

## Showcase data

## Splitting dataset

In [6]:
test_path = []
test_letters = []
#test_images = []
train_path = []
train_letters = []
#train_images = []
for i in range(len(path_list)):
    if (i % 3 == 0):
        test_path.append(path_list[i])
        test_letters.append(letter[i])
        #test_images = io.imread(path_list[i])
    else:
        train_path.append(path_list[i])
        train_letters.append(letter[i])
        #train_images = io.imread(path_list[i])

# Building Dataset

In [7]:
lettersIndex = ['А', 'Б', 'В', 'Г', 'Ґ', 'Д', 'Е', 'Є', 'Ж', 'З', 'И', 'І', 'Ї', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ч', 'Ц', 'Ч', 'Ш', 'Щ', 'Ь', 'Ю', 'Я']

In [8]:
from skimage import io, transform

class MNISTCyrDataset(torch.utils.data.Dataset):
    """MNIST cyrillics dataset"""

    def __init__(self, file_paths, file_letters, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.file_paths = file_paths
        self.file_letters = file_letters
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.file_paths[idx]
        image = io.imread(img_name)
        
        sample = [image, lettersIndex.index(self.file_letters[idx])]
        if self.transform:
            sample[0] = self.transform(sample[0])

        return sample

In [9]:

test_dataset = MNISTCyrDataset(file_paths=test_path, file_letters=test_letters, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,)),
                               torchvision.transforms.Grayscale(1),
                               torchvision.transforms.Resize((32, 32))
                             ]))

test_loader = torch.utils.data.DataLoader(test_dataset, 32, shuffle=True)

In [10]:
train_dataset = MNISTCyrDataset(file_paths=train_path, file_letters=train_letters, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,)),
                               torchvision.transforms.Grayscale(1),
                               torchvision.transforms.Resize((32, 32))
                             ]))

train_loader = torch.utils.data.DataLoader(test_dataset, 32, shuffle=True)

In [11]:
image , label = train_dataset[32]
image.size()

torch.Size([1, 32, 32])

# Building Network

In [12]:
class ConvolutionNNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionNNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(1024, 1000)
        self.fc2 = nn.Linear(1000, 34)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [13]:
# Train the model
num_epochs = 32
learning_rate = 0.001
total_step = len(train_loader)
loss_list = []
acc_list = []
model = ConvolutionNNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate);
criterion = nn.CrossEntropyLoss();

In [14]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/32], Step [100/2698], Loss: 2.5608, Accuracy: 31.25%
Epoch [1/32], Step [200/2698], Loss: 1.7528, Accuracy: 56.25%
Epoch [1/32], Step [300/2698], Loss: 1.5191, Accuracy: 59.38%
Epoch [1/32], Step [400/2698], Loss: 1.1335, Accuracy: 68.75%
Epoch [1/32], Step [500/2698], Loss: 1.2454, Accuracy: 62.50%
Epoch [1/32], Step [600/2698], Loss: 1.4338, Accuracy: 71.88%
Epoch [1/32], Step [700/2698], Loss: 1.5755, Accuracy: 71.88%
Epoch [1/32], Step [800/2698], Loss: 1.5328, Accuracy: 78.12%
Epoch [1/32], Step [900/2698], Loss: 1.3951, Accuracy: 62.50%
Epoch [1/32], Step [1000/2698], Loss: 1.1683, Accuracy: 90.62%
Epoch [1/32], Step [1100/2698], Loss: 1.5648, Accuracy: 62.50%
Epoch [1/32], Step [1200/2698], Loss: 1.4714, Accuracy: 71.88%
Epoch [1/32], Step [1300/2698], Loss: 1.0990, Accuracy: 75.00%
Epoch [1/32], Step [1400/2698], Loss: 0.8496, Accuracy: 87.50%
Epoch [1/32], Step [1500/2698], Loss: 1.1100, Accuracy: 71.88%
Epoch [1/32], Step [1600/2698], Loss: 0.7552, Accuracy: 84.38%
E

KeyboardInterrupt: 